In [ ]:
!pip install pennylane torch torchio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
shared_folder_path = '/content/drive/MyDrive/Quantum/'
!unzip {shared_folder_path + "mri_simple.zip"}

Archive:  /content/drive/MyDrive/Quantum/mri_simple.zip
replace Combined Dataset/test/Mild Impairment/1 (10).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Combined Dataset/test/Mild Impairment/1 (11).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: nA
replace Combined Dataset/test/Mild Impairment/1 (14).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
transform = transforms.Compose([
    transforms.Grayscale(),                # Ensure 1 channel
    transforms.Resize((64, 64)),         # Ensure consistent size
    transforms.ToTensor(),                 # Converts to shape (1, 128, 128)
])

train_dataset = datasets.ImageFolder('/content/Combined Dataset/train', transform=transform)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Step 1: Load all data from dataset and move to CUDA
all_images = []
all_labels = []

for img, label in train_dataset:
    all_images.append(img.unsqueeze(0))  # add batch dim
    all_labels.append(torch.tensor([label]))

# Stack all samples into single tensors
all_images = torch.cat(all_images).to(device)  # shape: (N, C, H, W)
all_labels = torch.cat(all_labels).to(device)  # shape: (N,)

# Step 2: Create TensorDataset on CUDA tensors
cuda_dataset = TensorDataset(all_images, all_labels)

# Step 3: Create DataLoader from cuda_dataset
train_loader = DataLoader(cuda_dataset, batch_size=24, shuffle=False)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
# show train_loader shape
for images, labels in train_loader:
    print(images.shape)
    print(labels)
    break

torch.Size([24, 1, 64, 64])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pennylane as qml
import numpy as np
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Set device
device = qml.device('lightning.gpu', wires=8, batch_obs=True)
# device = qml.device('default.qubit', wires=8)

# Manual implementation of ZZFeatureMap
def ZZFeatureMap(x, wires, reps=2):
    """
    Manual implementation of ZZFeatureMap encoding
    Args:
        x: input data (flattened)
        wires: list of wire indices
        reps: number of repetitions
    """
    n_qubits = len(wires)

    for rep in range(reps):
        # Single qubit rotations
        for i, wire in enumerate(wires):
            if i < len(x):
                qml.RZ(x[i], wires=wire)

        # Entangling gates (ZZ interactions)
        for i in range(n_qubits):
            for j in range(i + 1, n_qubits):
                if i < len(x) and j < len(x):
                    # ZZ interaction: exp(-i * x[i] * x[j] * Z_i * Z_j)
                    qml.CNOT(wires=[wires[i], wires[j]])
                    qml.RZ(x[i] * x[j], wires=wires[j])
                    qml.CNOT(wires=[wires[i], wires[j]])

# Convolutional ansatz U_SO4
def U_SO4(params, wires, noise_model='noiseless', noise_strength=0.0):
    """
    SO4 ansatz for convolutional layer
    """
    qml.RY(params[0], wires=wires[0])
    qml.RY(params[1], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[2], wires=wires[0])
    qml.RY(params[3], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RY(params[4], wires=wires[0])
    qml.RY(params[5], wires=wires[1])

# Pooling ansatz
def Pooling_ansatz1(params, wires):
    """
    Pooling ansatz for pooling layer
    """
    qml.CRZ(params[0], wires=[wires[0], wires[1]])
    qml.PauliX(wires=wires[0])
    qml.CRX(params[1], wires=[wires[0], wires[1]])

# Conv and pooling layers
def conv_layer(U, params, wires_list):
    """Apply convolution layer"""
    for wires in wires_list:
        U(params, wires=wires)

def pooling_layer(V, params, wires_list):
    """Apply pooling layer"""
    for i, wires in enumerate(wires_list):
        V(params[i*2:(i+1)*2], wires=wires)

# Improved QCNN structure
def qcnn_structure(U_fn, params, U_params, num_classes):
    """
    Improved QCNN structure with better parameter management
    """
    num_measurement_qubits = int(np.ceil(np.log2(num_classes)))
    param_idx = 0

    # First conv + pooling layer (8 -> 4 qubits)
    conv_layer(U_fn, params[param_idx:param_idx + U_params],
               [[0, 1], [2, 3], [4, 5], [6, 7]])
    param_idx += U_params

    pooling_layer(Pooling_ansatz1, params[param_idx:param_idx + 8],
                  [[1, 0], [3, 2], [5, 4], [7, 6]])
    param_idx += 8

    active_wires = [0, 2, 4, 6]
    if num_measurement_qubits >= 3:
        return active_wires[:int(num_measurement_qubits)], param_idx

    # Second conv + pooling layer (4 -> 2 qubits)
    conv_layer(U_fn, params[param_idx:param_idx + U_params],
               [[0, 2], [4, 6]])
    param_idx += U_params

    pooling_layer(Pooling_ansatz1, params[param_idx:param_idx + 4],
                  [[2, 0], [6, 4]])
    param_idx += 4

    active_wires = [0, 4]
    if num_measurement_qubits == 2:
        return active_wires, param_idx

    # Third conv + pooling layer (2 -> 1 qubit)
    conv_layer(U_fn, params[param_idx:param_idx + U_params],
               [[0, 4]])
    param_idx += U_params

    pooling_layer(Pooling_ansatz1, params[param_idx:param_idx + 2],
                  [[4, 0]])
    param_idx += 2

    return [0], param_idx

# Calculate total parameters needed
def calculate_total_params(num_classes):
    """Calculate total number of parameters needed for the QCNN"""
    U_params = 6  # U_SO4 has 6 parameters
    total_params = 0

    # First layer: 4 conv blocks + 4 pooling blocks
    total_params += U_params + 8

    # Second layer: 2 conv blocks + 2 pooling blocks
    total_params += U_params + 4

    # Third layer: 1 conv block + 1 pooling block
    total_params += U_params + 2

    return total_params

# Quantum circuit
@qml.qnode(device, interface='torch', diff_method='adjoint')
def quantum_circuit(inputs, params, num_classes):
    """
    Complete quantum circuit with encoding and QCNN structure
    """
    # Convert inputs to float32 to ensure compatibility
    inputs = inputs.float()
    # Data encoding using ZZFeatureMap
    ZZFeatureMap(inputs, wires=range(8), reps=1)

    # Apply QCNN structure
    measurement_wires, _ = qcnn_structure(U_SO4, params, 6, num_classes)

    # Measurements
    if len(measurement_wires) == 1:
        return qml.expval(qml.PauliZ(measurement_wires[0]))
    else:
        return [qml.expval(qml.PauliZ(wire)) for wire in measurement_wires]

# Hybrid quantum-classical model
class QCNNModel(nn.Module):
    def __init__(self, num_classes=2, input_size=64*64):
        super(QCNNModel, self).__init__()
        self.num_classes = num_classes
        self.input_size = input_size

        # Calculate quantum parameters
        self.num_qparams = calculate_total_params(num_classes)

        # Classical preprocessing layers (fixed BatchNorm issue)
        self.classical_layers = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 8),  # Reduce to 8 features for 8 qubits
            nn.Tanh()  # Bounded activation for quantum circuit
        )

        # Quantum parameters (better initialization)
        self.qparams = nn.Parameter(torch.randn(self.num_qparams, dtype=torch.float32) * 0.1, requires_grad=True)

        # Classical post-processing
        num_quantum_outputs = int(np.ceil(np.log2(num_classes))) if num_classes > 1 else 1
        if num_quantum_outputs == 1:
            self.post_processing = nn.Sequential(
                nn.Linear(1, 16),
                nn.ReLU(),
                nn.Linear(16, num_classes)
            )
        else:
            self.post_processing = nn.Sequential(
                nn.Linear(num_quantum_outputs, 16),
                nn.ReLU(),
                nn.Linear(16, num_classes)
            )

    def forward(self, x):
        batch_size = x.shape[0]
        x = x.view(batch_size, -1).float()  # Flatten and ensure float32

        # Classical preprocessing
        x = self.classical_layers(x)

        # Process each sample in the batch
        quantum_outputs = []
        for i in range(batch_size):
            # Use processed features directly (already bounded by Tanh)
            sample = x[i]

            # Quantum circuit
            q_out = quantum_circuit(sample, self.qparams, self.num_classes)

            if isinstance(q_out, list):
                quantum_outputs.append(torch.stack(q_out))
            else:
                quantum_outputs.append(q_out.unsqueeze(0))

        quantum_output = torch.stack(quantum_outputs).float()  # Ensure float32

        # Classical post-processing
        output = self.post_processing(quantum_output)

        return output

# Visualization function
def visualize_dataset_samples(dataset, num_samples=8, figsize=(12, 8)):
    """
    Visualize sample images from the dataset
    """
    fig, axes = plt.subplots(2, 4, figsize=figsize)
    axes = axes.ravel()

    # Get class names
    class_names = dataset.classes if hasattr(dataset, 'classes') else [f'Class {i}' for i in range(len(dataset.class_to_idx))]

    for i in range(min(num_samples, len(dataset))):
        # Get random sample
        idx = np.random.randint(0, len(dataset))
        image, label = dataset[idx]

        # Convert tensor to numpy and handle single channel
        if image.dim() == 3:
            img_np = image.squeeze().numpy()
        else:
            img_np = image.numpy()

        axes[i].imshow(img_np, cmap='gray')
        axes[i].set_title(f'{class_names[label]}')
        axes[i].axis('off')

    # Hide unused subplots
    for i in range(num_samples, len(axes)):
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Dataset analysis function
def analyze_dataset(dataset):
    """
    Analyze dataset distribution and properties
    """
    print("=== Dataset Analysis ===")
    print(f"Total samples: {len(dataset)}")

    if hasattr(dataset, 'classes'):
        print(f"Number of classes: {len(dataset.classes)}")
        print(f"Class names: {dataset.classes}")

        # Count samples per class
        from collections import Counter
        labels = [dataset[i][1] for i in range(len(dataset))]
        class_counts = Counter(labels)

        print("Class distribution:")
        for i, class_name in enumerate(dataset.classes):
            count = class_counts.get(i, 0)
            percentage = (count / len(dataset)) * 100
            print(f"  {class_name}: {count} samples ({percentage:.1f}%)")

    # Sample image info
    sample_image, sample_label = dataset[0]
    print(f"\nSample image shape: {sample_image.shape}")
    print(f"Image dtype: {sample_image.dtype}")
    print(f"Value range: [{sample_image.min():.3f}, {sample_image.max():.3f}]")

    return len(dataset.classes) if hasattr(dataset, 'classes') else None
def train_qcnn(model, train_loader, num_epochs=10, learning_rate=0.01):
    """
    Training function for QCNN model with improved optimization
    """
    criterion = nn.CrossEntropyLoss()

    # Separate optimizers for classical and quantum parameters
    classical_params = [p for name, p in model.named_parameters() if 'qparams' not in name]
    quantum_params = [model.qparams]

    classical_optimizer = optim.Adam(classical_params, lr=learning_rate)
    quantum_optimizer = optim.Adam(quantum_params, lr=learning_rate * 10)  # Higher LR for quantum

    # Learning rate schedulers
    classical_scheduler = optim.lr_scheduler.StepLR(classical_optimizer, step_size=3, gamma=0.7)
    quantum_scheduler = optim.lr_scheduler.StepLR(quantum_optimizer, step_size=3, gamma=0.7)

    model.train()
    losses = []
    accuracies = []

    # Get number of classes from model
    num_classes = model.num_classes

    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(train_loader):
            # Validate targets are within valid range
            if torch.any(target >= num_classes) or torch.any(target < 0):
                print(f"Warning: Invalid target values found. Expected range [0, {num_classes-1}], got {target}")
                # Clip targets to valid range
                target = torch.clamp(target, 0, num_classes-1)

            classical_optimizer.zero_grad()
            quantum_optimizer.zero_grad()

            # Forward pass
            output = model(data)
            loss = criterion(output, target)

            # Backward pass
            loss.backward()

            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            classical_optimizer.step()
            quantum_optimizer.step()

            # Statistics
            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            if batch_idx % 10 == 0:  # Less frequent printing
                print(f'Epoch: {epoch+1}/{num_epochs}, Batch: {batch_idx}, '
                      f'Loss: {loss.item():.4f}')

        # Step schedulers
        classical_scheduler.step()
        quantum_scheduler.step()

        epoch_loss = total_loss / len(train_loader)
        epoch_acc = 100. * correct / total

        losses.append(epoch_loss)
        accuracies.append(epoch_acc)

        print(f'Epoch: {epoch+1}/{num_epochs}, '
              f'Average Loss: {epoch_loss:.4f}, '
              f'Accuracy: {epoch_acc:.2f}%')
        print(f'Classical LR: {classical_scheduler.get_last_lr()[0]:.6f}, '
              f'Quantum LR: {quantum_scheduler.get_last_lr()[0]:.6f}')

    return losses, accuracies



In [ ]:
# Initialize model
num_classes = 4  # Sesuaikan dengan jumlah kelas dalam dataset Anda
model = QCNNModel(num_classes=num_classes).to('cuda')

print(f"Model initialized with {sum(p.numel() for p in model.parameters())} parameters")
print(f"Quantum parameters: {model.num_qparams}")

# Training (uncomment untuk training dengan data asli)
print("train_loader", train_loader.dataset[0][0].device)
losses, accuracies = train_qcnn(model, train_loader, num_epochs=5, learning_rate=0.01)

print("QCNN model berhasil dibuat!")
print("Untuk menggunakan dengan dataset Anda, uncomment bagian train_dataset dan train_loader")
torch.save(model.state_dict(), shared_folder_path + "qcnn.pth")

In [ ]:
# save the model to the drive
torch.save(model.state_dict(), shared_folder_path + "qcnn.pth")